In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
import requests
from html_table_parser import parser_functions

In [5]:
product_list = list(pd.read_csv("../data/product_list.csv")['product_label'])
print('Total Number of Products:',len(product_list))

Total Number of Products: 77001


In [4]:
product_name = []
ingredient_lst = []
formatted_ingredient_lst = []
what_lst = []

CRAWLING - REMARK<br>
제품페이지로 접근을 시도할 제품 총 77001개
&rarr; 데이터 수집에 상당한 시간이 소요 (24시간 이상 예상)

아래 소스코드를 사용하되, 효율적인 수집을 위하여 다음과 같이 진행함:<br>
제품리스트(product_list)를 분할 (list slicing) &rarr; 다수의 컴퓨터에서 동시 수집하여 저장 후 병합

In [ ]:
for product in tqdm_notebook(product_list):
    url = f"https://incidecoder.com/products/{product}"
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

    data = soup.find('table',{"class" : "product-skim fs16"})
    
    ingtmp = []
    for tag in data.find_all('a', {'class': "black ingred-detail-link"}):
        ingtmp.append(tag.attrs['href'][13:])
    formatted_ingredient_lst.append(ingtmp)
    
    
    df = parser_functions.make2d(data)[1:]
    tmpdf = pd.DataFrame(df, columns=["Ingredient name", "what-it-does", "irr., com.", "ID-Rating"])
    
    Bold_lst = []
    for ing in data.find_all('td', {'class': 'bold'}):
        Bold_lst.append(ing.text.replace('\n','').strip())
    
    indexes = []
    for stop in Bold_lst:
        indexes.append(tmpdf[tmpdf['Ingredient name'] == stop].index[0])
    
    tmpdf.drop(indexes, axis=0,inplace=True)

    ingredient_lst.append(list(tmpdf['Ingredient name']))
    what_lst.append(list(tmpdf["what-it-does"]))
    product_name.append(product)

In [ ]:
each_ingredient_lst = []
for lst in ingredient_lst:
    for ing in lst:
        each_ingredient_lst.append(ing)

each_formatted_ingredient_lst = []
for lst in formatted_ingredient_lst:
    for ing in lst:
        each_formatted_ingredient_lst.append(ing)

each_what_lst = []
for lst in what_lst:
    for does in lst:
        tmp = []
        for does in does.replace('\n','').replace('\u200b','').split(','):
            tmp.append(does.strip())
        each_what_lst.append(tmp)

최종 제품데이터 (제품명 - 성분리스트 - 성분리스트(formatted))

In [ ]:
cols = ['product name', 'ingredients', 'formatted ingredients']
product_df = pd.DataFrame(columns=cols)

product_df['product name'] = product_name
product_df['ingredients'] = ingredient_lst
product_df['formatted ingredients'] = formatted_ingredient_lst

product_df

In [ ]:
product_df.to_csv('../data/inci_prod_final.csv',index=False)


||product name|ingredients|formatted ingredients|
|---|---|---|---|
|0|innisfree-auto-eyebrow-pencil-no-4|[Hydrogenated Soybean Oil, Hydrogenated Coco-G...|[hydrogenated-soybean-oil, hydrogenated-coco-g...|
|1|bielenda-good-skin-acid-peel-micro-exfoliating...|[Aqua (Water), Mandelic Acid, Lactobionic Acid...|[water, mandelic-acid, lactobionic-acid, niaci...|
|2|neals-yard-remedies-st-johns-wort-macerated-oil|[Olea Europaea (Olive) Fruit Oil*, Hypericum P...|[olea-europaea-fruit-oil, hypericum-perforatum...|
|...|...|...|...|

최종 성분데이터 (성분명 - 성분명(formatted) - 효능(what-it-does))

In [ ]:
ingredient_df = pd.DataFrame(columns=['ingredients','formatted ingredients','what-it-does'])
ingredient_df['ingredients'] = each_ingredient_lst
ingredient_df['formatted ingredients'] = each_formatted_ingredient_lst
ingredient_df['what-it-does'] = each_what_lst
ingredient_df

In [ ]:
ingredient_df.to_csv('../inci_ing_final.csv',index=False)

||ingredients|formatted ingredients|what-it-does|
|---|---|---|---|
|0|Hydrogenated Soybean Oil|hydrogenated-soybean-oil|[emollient, moisturizer/humectant, viscosity c...|
|1|Hydrogenated Coco-Glycerides|hydrogenated-coco-glycerides|[emollient]|
|2|Iron Oxides (Ci 77499)|iron-oxides|[colorant]|